# Setup Environment

## Python Env

In [1]:
!pip show torch

Name: torch
Version: 2.9.1+cu126
Summary: Tensors and Dynamic neural networks in Python with strong GPU acceleration
Home-page: https://pytorch.org
Author: 
Author-email: PyTorch Team <packages@pytorch.org>
License: BSD-3-Clause
Location: c:\users\admin\.conda\envs\conformal\lib\site-packages
Requires: filelock, fsspec, jinja2, networkx, sympy, typing-extensions
Required-by: torchvision


In [2]:
!pip show torchvision

Name: torchvision
Version: 0.24.1+cu126
Summary: image and video datasets and models for torch deep learning
Home-page: https://github.com/pytorch/vision
Author: PyTorch Core Team
Author-email: soumith@pytorch.org
License: BSD
Location: c:\users\admin\.conda\envs\conformal\lib\site-packages
Requires: numpy, pillow, torch
Required-by: 


In [3]:
# !pip install pydicom onnxruntime-gpu albumentations

In [4]:
# !pip install matplotlib numpy onnxruntime-gpu pandas tqdm pillow spicy scikit-image==0.21.0 scikit-learn==1.3.2 imantics feret pydicom python-dotenv albumentations

## Drive Mountpoint

## Libraries

In [5]:
%cd ..

e:\AwesomeConformalPrediction


c:\Users\Admin\.conda\envs\conformal\lib\site-packages\IPython\core\magics\osm.py:417: UserWarning: This is now an optional IPython functionality, setting dhist requires you to install the `pickleshare` library.
  self.shell.db['dhist'] = compress_dhist(dhist)[-100:]


In [6]:
from typing import Tuple, List
import os
import cv2
import csv
import json
import torch
import shutil
import joblib
import pydicom
import zipfile
import argparse
import numpy as np
import pandas as pd
import torch.nn as nn
from copy import deepcopy
from tqdm import tqdm
from pathlib import Path
# import onnxruntime as ort
import albumentations as A
import torch.nn.functional as F

from PIL import Image, ImageDraw, ImageFont

import warnings
warnings.filterwarnings("ignore")

In [7]:
# import rootutils 
# rootutils.set_root(os.getcwd(), pythonpath=True)
# from src.models.model_2D.segcls.segcls_module import SegClsModule
# from src.models.model_2D.segcls.net.segcls import SegClsNet
# from src.models.model_2D.segclsnet import segcls
# import sys
# sys.modules['src.models.model_2D.segcls.net.segcls.SegClsNet'] = SegClsNet

# Model

## Model Class

In [8]:
CLS_LUNG_POS_THRESHOLD = [0.35, 0.75, 0.5]

In [9]:
dict_columns = {
    "slice_info_columns": ["bnid", "raw_id", "pixel_spacing_w", "pixel_spacing_h", "intercept", "slope", "window_center", "window_width"],
    "slice_info_columns_short": ["code", "slice_id", "raw_id"],
    "nodule": {
        "dam_do": ["Nhóm 1 - Đậm độ - 1.1 Đặc", "Nhóm 1 - Đậm độ - 1.2 Bán đặc", "Nhóm 1 - Đậm độ - 1.3 Kính mờ", "Nhóm 1 - Đậm độ - 1.4 Đông đặc"],
        "voi_hoa": ["Nhóm 2 - Đậm độ vôi - 2.1 Không có vôi", "Nhóm 2 - Đậm độ vôi - 2.2 Có vôi"],
        "chua_mo": ["Nhóm 3 - Đậm độ mỡ - 3.1 Không chứa mỡ", "Nhóm 3 - Đậm độ mỡ - 3.2 Có chứa mỡ"],
        "duong_vien": ["Nhóm 4 - Bờ và Đường viền - 4.1 Tròn đều", "Nhóm 4 - Bờ và Đường viền - 4.2 Đa thuỳ", "Nhóm 4 - Bờ và Đường viền - 4.3 Bờ không đều", "Nhóm 4 - Bờ và Đường viền - 4.4 Tua gai"],
        "tao_hang": ["Nhóm 5 - Tạo hang - 5.1 Không có", "Nhóm 5 - Tạo hang - 5.2 Hang lành tính", "Nhóm 5 - Tạo hang - 5.3 Hang ác tính"],
        "di_can": ["Nhóm 6 - Di căn phổi - 6.1 Di căn cùng bên", "Nhóm 6 - Di căn phổi - 6.0 Không di căn", "Nhóm 6 - Di căn phổi - 6.2 Di căn đối bên"],
    },
    "cls_nodule_group": {
        "Nhóm 1 - Đậm độ": ['1.1 Đặc', '1.2 Bán đặc', '1.3 Kính mờ', '1.4 Đông đặc'],
        "Nhóm 2 - Đậm độ vôi": ['2.1 Không có vôi', '2.2 Có vôi'],
        "Nhóm 3 - Đậm độ mỡ": ['3.1 Không chứa mỡ', '3.2 Có chứa mỡ'],
        "Nhóm 4 - Bờ và Đường viền": ['4.1 Tròn đều', '4.2 Đa thuỳ', '4.3 Bờ không đều', '4.4 Tua gai'],
        "Nhóm 5 - Tạo hang": ['5.1 Không có', '5.2 Hang lành tính', '5.3 Hang ác tính'],
        "Nhóm 6 - Di căn phổi": ['6.1 Di căn cùng bên', '6.0 Không di căn', '6.2 Di căn đối bên']
    },
    "bbox": ["left", "top", "width", "height"],
    "lung_pos": ["right_lung", "left_lung"],
    "lung_loc": ["Vị trí giải phẫu - 1. Thuỳ trên phải", "Vị trí giải phẫu - 2. Thuỳ giữa phải", "Vị trí giải phẫu - 3. Thuỳ dưới phải", "Vị trí giải phẫu - 4. Thuỳ trên trái", "Vị trí giải phẫu - 5. Thuỳ dưới trái"],
    "lung_damage": {
        "dong_dac": ["not_dong_dac", "Tổn thương viêm - 1. Đông đặc"],
        "kinh_mo": ["not_kinh_mo", "Tổn thương viêm - 2. Kính mờ"],
        "phe_quan_do": ["not_phe_quan_do", "Tổn thương viêm - 3. Hình phế quản đồ"],
        "nu_tren_canh": ["not_nu_tren_canh", "Tổn thương viêm - 4. Nốt mờ dạng nụ trên cành"],
    },
    "cls_lung_damage_group": {
        "Tổn thương viêm - 1. Đông đặc": ["not_dong_dac", "Tổn thương viêm - 1. Đông đặc"],
        "Tổn thương viêm - 2. Kính mờ": ["not_kinh_mo", "Tổn thương viêm - 2. Kính mờ"],
        "Tổn thương viêm - 3. Hình phế quản đồ": ["not_phe_quan_do", "Tổn thương viêm - 3. Hình phế quản đồ"],
        "Tổn thương viêm - 4. Nốt mờ dạng nụ trên cành": ["not_nu_tren_canh", "Tổn thương viêm - 4. Nốt mờ dạng nụ trên cành"],
    }
}

In [10]:
# Helper
def find_bbox(mask: np.array, ratio: bool = False, ssize = [1, 1]):
    """_summary_
    Args:
        mask (np.ndarray): (h, w) - np.uint8
    Returns:
        List[Tuple[int, int, int, int]]: List of bounding boxes
    """
    W, H = ssize
    mask = (mask > 0.5).astype(np.uint8)
    contours, _ = cv2.findContours(
        mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    cnt_bboxes = []
    for contour in contours: 
        (x, y, w, h) = cv2.boundingRect(contour)
        if ratio:
            cnt_bboxes.append((x / W, y / H, w / W, h / H))
        else:
            cnt_bboxes.append((x, y, w, h))
    return contours, cnt_bboxes

def get_bbox(mask: np.ndarray, min_area: int):
    def check_tiny_nodule(bbox: Tuple[int, int, int, int], min_area: int = 25) -> bool:
        """_summary_
        Args:
            bbox (Tuple[int, int, int, int]): (x, y, w, h)
            min_area (int, optional): Defaults to 25.
        Returns:
            bool: True if bbox area < min_area
        """
        return bbox[2] * bbox[3] < min_area

    _, bboxes = find_bbox(mask[0])
    filtered_bboxes = []
    for bbox in bboxes:
        # filter tiny nodule: min_area=25 for image (512x512)
        if check_tiny_nodule(bbox, min_area):
            x, y, w, h = bbox
            mask[0, y: y+h, x: x+w] = 0
        else:
            filtered_bboxes.append(bbox)

    return mask, filtered_bboxes

def lung_loc_postprocess(lung_loc_mask):
    for i in range(lung_loc_mask.shape[-1]):
        mask = lung_loc_mask[:, :, i]
        contours, _ = find_bbox(mask)
        if len(contours):
            max_contour = max(contours, key=cv2.contourArea)
            mask = np.zeros_like(mask)
            cv2.drawContours(mask, [max_contour], -1,
                             (255), thickness=cv2.FILLED)
            lung_loc_mask[:, :, i] = (
                np.array(mask) / 255.0).astype(lung_loc_mask.dtype)

    return lung_loc_mask


In [11]:
# class KCCTPipeline(nn.Module):
#   def __init__(self,
#               net: nn.Module,
#               output_dir: str,
#               device: str,
#               use_fp16: bool = True,
#               debug: bool = False) -> None:

#     super().__init__()

#     self.debug = debug
#     self.device = device
#     self.net: SegClsNet = net
#     self.net.to(device)
#     self.seg_net = self.net.seg_net
#     self.cls_net = self.net.cls_nodule_net
#     print("", "-> Infer Nodule Model loaded!")
#     self.original_size = (512, 512)
#     self.preprocessed_size = (352, 352)
#     self.output_dir = output_dir

#     self.mask_dir = self.output_dir / "mask"
#     self.csv_dir = self.output_dir / "csv"
#     self.logit_dir = self.output_dir / "logit"

#     self.mask_dir.mkdir(parents=True, exist_ok=True)
#     self.csv_dir.mkdir(parents=True, exist_ok=True)
#     self.logit_dir.mkdir(parents=True, exist_ok=True)

#     df_columns = deepcopy(dict_columns["slice_info_columns_short"])
#     df_columns.extend(["mask_id"] + dict_columns["lung_pos"])
#     df_columns.extend(dict_columns["bbox"])
#     for _, group_cols in dict_columns["lung_damage"].items():
#         df_columns.extend(group_cols)
#     for group, group_cols in dict_columns["nodule"].items():
#         df_columns.extend(group_cols)
#     self.df_columns = df_columns
#     self.init_transform()

#     # Initialized
#     print()

#   # Helper & process
#   def init_transform(self):
#       bbox_params = A.BboxParams(
#           format="coco", label_fields=["class_labels"])
#       self.transform_img = A.Compose(
#           transforms=[A.Resize(*self.preprocessed_size, p=1)])
#       self.rev_transform_mask = A.Compose(
#           transforms=[A.Resize(*self.original_size, p=1)])
#       self.rev_transform_mask_box = A.Compose(transforms=[A.Resize(*self.original_size, p=1)],
#                                               bbox_params=bbox_params)

#   def save_csv(self, bnid: str, res_slice: List[List]):
#       csv_path = self.csv_dir / f"{bnid}.csv"
#     #   print(len(res_slice[0]), len(self.df_columns))
#       if not csv_path.exists():
#           # Create a new CSV file with the specified columns and save it
#           df = pd.DataFrame(res_slice, columns=self.df_columns)
#           df.to_csv(csv_path, index=False)
#       else:
#           # Append new rows directly to the existing CSV file
#           with open(csv_path, 'a', newline='') as f:
#               writer = csv.writer(f)
#               writer.writerows(res_slice)

#   def ct_normalize(self, image: np.ndarray) -> np.ndarray:
#       """_summary_: normalization
#       Args:
#           image (np.ndarray): (h, w) or (c, h, w) or (b, c, h, w)
#       Returns:
#           np.ndarray: (h, w) or (c, h, w) or (b, c, h, w)
#       """
#       if len(image.shape) == 4:
#           min_val = image.amin(dim=(1, 2, 3), keepdim=True)
#           max_val = image.amax(dim=(1, 2, 3), keepdim=True)
#       else:
#           min_val = image.min()
#           max_val = image.max()

#       if (max_val - min_val).item == 0:
#           return image

#       return (image - min_val) / (max_val - min_val)

#   def preprocess(self, raw_images: np.ndarray) -> np.ndarray:
#       """_summary_: down-sampling
#       Args:
#           raw_images (np.ndarray): (512, 512)
#       Returns:
#           np.ndarray: (1, 1, 352, 352)
#       """

#       norm_images = self.ct_normalize(raw_images)
#       transformed = self.transform_img(image=norm_images)
#       downsampled_images = transformed["image"]
#       return torch.tensor(downsampled_images[None, None, ...], device=self.device)

#   def seg_postprocess(self, mask: np.ndarray, bboxes: np.ndarray = np.array([])) -> Tuple[np.ndarray, np.ndarray]:
#       """_summary_
#       Args:
#           mask (np.ndarray): (1, 1, 352, 352)
#           bboxes (np.ndarray): (n, 4)
#       Returns:
#           Tuple[np.ndarray, np.ndarray]: [(512, 512), (n, 4)]
#       """
#       mask = (mask > 0.5).astype(np.uint8)  # (1, 1, w, h) -> (1, w, h)
#       if mask.shape[0] == 1:  # binary mask
#           mask = mask[0]
#           if len(bboxes):
#               bboxes = bboxes.astype(np.float32)
#               fake_label = list(range(bboxes.shape[0]))
#               transformed = self.rev_transform_mask_box(image=mask,
#                                                         bboxes=bboxes, class_labels=fake_label)
#               bboxes = transformed["bboxes"].astype(np.int64)
#           else:
#               transformed = self.rev_transform_mask(image=mask)

#           upsampled_mask = transformed["image"]
#           return upsampled_mask, bboxes

#       elif mask.shape[0] == 6:
#           upsampled_mask = [
#               cv2.resize(mask[i], self.original_size, interpolation=cv2.INTER_LINEAR) for i in range(mask.shape[0])
#           ]
#           upsampled_mask = np.stack(upsampled_mask, axis=-1)
#           return upsampled_mask

#   def cls_postprocess(self, logits: List[np.ndarray]):
#       def softmax(logit, axis: int = None):
#           if isinstance(logit, torch.Tensor):
#             #   print(logit.shape)
#               return F.softmax(logit, dim=axis).cpu().numpy()
#           exp_logit = np.exp(logit - np.max(logit, axis=axis, keepdims=True))
#           return exp_logit / np.sum(exp_logit, axis=axis, keepdims=True)
     
#       probs = [softmax(logit, axis=1) for logit in logits]
#       return probs

#   # modified postprocess
#   def ort_postprocess(self, ort_outs):
#       out_types = ort_outs.keys()
#       probs = ort_outs['seg_nodule'] if isinstance(ort_outs['seg_nodule'], torch.Tensor) else torch.from_numpy(ort_outs['seg_nodule'])
#       pred_masks = (probs > 0.5).to(torch.uint8)
#       ort_outs['seg_nodule'] = pred_masks.numpy()

#       if 'seg_lung_loc' in out_types:
#           probs = ort_outs['seg_lung_loc'] if isinstance(ort_outs['seg_lung_loc'], torch.Tensor) else torch.from_numpy(ort_outs['seg_lung_loc'])
#           pred_masks = torch.argmax(probs, dim=1)
#           pred_masks = F.one_hot(pred_masks, num_classes=probs.shape[1]).permute(0, 3, 1, 2)
#           ort_outs['seg_nodule'] = pred_masks.numpy()
#       return ort_outs

#   # Infer 2D
#   @torch.no_grad()
#   def infer_2D(self, raw_ct_image: np.array, bnid: str, slice_id: str, raw_id: str, pixel_spacing: np.ndarray, intercept, slope, window_center, window_width, label_bboxes=None, pbar: tqdm=None):
#       """_summary_: infer2D for one slice
#       Args:
#           raw_ct_image (np.array): (512, 512)
#       """
#       csv_path = self.csv_dir / f"{bnid}.csv"
#       # bnid_dir = self.raw_image_dir / bnid
#       if csv_path.exists():
#           df_case = pd.read_csv(csv_path)
#         #   print(raw_id, df_case["raw_id"])
#           if raw_id in df_case["raw_id"].values:
#             if pbar: 
#                 pbar.write(f"{bnid}-{raw_id} infered".ljust(30))
#             else:
#                 print(f"{bnid} - {raw_id} is infered")
#             return
#       # else:
#           # bnid_dir.mkdir(parents=True, exist_ok=True)

#       # np.save(bnid_dir / f"{raw_id}.npy", raw_ct_image)

#       preprocessed_images = self.preprocess(raw_ct_image.astype(np.float32))
#       seg_outputs = self.net.seg_forward(preprocessed_images)

#       seg_nodule = seg_outputs['seg_nodule'][0]  # remove batch
#       # seg_lung_loc = outputs[1][0] # remove batch
#       cls_lung_pos = seg_outputs['cls_lung_pos']  # remove batch
#       fm = seg_outputs['fm'][0]  # remove batch
#     #   print(fm.shape, fm)

#       seg_nodule, nodule_bboxes = get_bbox(seg_nodule.numpy(),
#                                             min_area=(25 * (seg_nodule.shape[1] / self.original_size[0]))**2)
#       if label_bboxes: 
#           nodule_bboxes = list(label_bboxes.values())
#         #   print(nodule_bboxes)
#           label_ids = list(label_bboxes.keys())
#       else: 
#           label_ids = None
     
#       nodules = []
#     #   fm = torch.nan_to_num(fm)
#       for (x, y, w, h) in nodule_bboxes:
#           if x < 1: 
#               W, H = fm.shape[1:]
#               (x, y, w, h) = (int(x * W), int(y * H), int(w * W), int(h * H))
#         #   print(x,y,w,h)
#           nodules.append(fm[:, y: y+h, x: x+w].mean(axis=(1, 2)))
#     #   print(nodules)
#       if len(nodules):
#             nodules = torch.stack(nodules, dim=0)
#             cls_outputs = None
#             for nodule in nodules:
#                 # not supported dynamic batch size
#                 output = self.cls_net(nodule.reshape(1, 1024))

#                 if cls_outputs == None:
#                     cls_outputs = output
#                 else:
#                     # cls_outputs = torch.cat((cls_outputs, output), dim=0)
#                     for id in range(len(output)):
#                         cls_outputs[id] = torch.cat((cls_outputs[id], output[id]), dim=0)

#             cls_nodule = cls_outputs
#             cls_nodule_prob = self.cls_postprocess(cls_nodule)
#             # print(cls_nodule_prob)

#       seg_nodule, nodule_bboxes = self.seg_postprocess(
#           seg_nodule, np.array(nodule_bboxes))
#       # seg_lung_loc = lung_loc_postprocess(self.seg_postprocess(seg_lung_loc))
#       cls_lung_pos_prob = self.cls_postprocess(cls_lung_pos)
#     #   print(cls_lung_pos_prob)

#       res_slice = [[bnid, slice_id, raw_id]]
#       # first dim: batch
#       # second dim: 0 for probability of outside, 1 for probability of inside
#       lung_pos_res = [val[0, 1] for val in cls_lung_pos_prob]
#       res_slice[0].extend(lung_pos_res)
#       # outside slice
#       if lung_pos_res[0] < CLS_LUNG_POS_THRESHOLD[0] and lung_pos_res[1] < CLS_LUNG_POS_THRESHOLD[0]:
#           seg_nodule = np.zeros_like(seg_nodule)  # clean mask
#           nodule_bboxes = []
#       # outside slice
#       # if lung_pos_res[0] < CLS_LUNG_POS_THRESHOLD[1] and lung_pos_res[1] < CLS_LUNG_POS_THRESHOLD[1]:
#       #     seg_lung_loc = np.zeros_like(seg_lung_loc) # clean mask

#       # nodule classification
#       slice_info, res_slice, nodule_id = res_slice[0], [], -1
#       for i, bbox in enumerate(nodule_bboxes):
#           if label_ids:
#               label_id = label_ids[i]
#           else:
#               label_id = None
#           _cls_nodule_prob = [val[i].tolist()[:-1]
#                               for val in cls_nodule_prob]
#           nodule_id += 1
#           nodule_info = [nodule_id if not label_id else label_id] + list(bbox) + sum(_cls_nodule_prob, [])
#           res_slice.append(slice_info + nodule_info)  
#       # no nodule -> clean slice
#       if len(res_slice) == 0:
#         #   _cls_nodule_prob = [-1] * 26
#         #   bbox = [-1] * len(dict_columns["bbox"])
#         #   no_nodule_info = [nodule_id] + bbox + _cls_nodule_prob
          
#         #   res_slice.append(slice_info + no_nodule_info)

#       if "logit" in seg_outputs.keys():
#           des_logit_dir = self.logit_dir / bnid
#           des_logit_dir.mkdir(parents=True, exist_ok=True)
#         #   print(seg_outputs['logit'].shape)
#           logit = F.sigmoid(seg_outputs['logit'])[0, 0].cpu().numpy()
#           img = (logit * 255).astype(np.uint8)
#           cv2.imwrite(des_logit_dir / f"{raw_id}.jpg", img)
#       else:
#           pbar.write("Logit is missing") if pbar is not None else print("Logit is missing")

#       if seg_outputs['seg_nodule'].sum() > 0:
#           # save nodule mask prediction
#           seg_nodule = seg_outputs['seg_nodule'][0, 0].cpu().numpy()
#           des_path_mask = self.mask_dir / bnid
#           des_path_mask.mkdir(parents=True, exist_ok=True)
#           pred_nodule_mask = (seg_nodule > 0.5).astype(np.uint8)
#           cv2.imwrite(des_path_mask /
#                       f"{raw_id}.jpg", pred_nodule_mask * 255)
#       else:
#           if pbar:
#             pbar.set_description_str(f"{bnid}-{raw_id} empty")
#           else:
#             pbar.write(f"{bnid}-{raw_id} empty seg")
#       # save lung location mask prediction
#       # des_path_mask = self.new_mask_dir / bnid
#       # des_path_mask.mkdir(parents=True, exist_ok=True)
#       # for i in range(1, seg_lung_loc.shape[-1]):
#       #     pred_lung_loc_mask = (seg_lung_loc[:,:,i] > 0.5).astype(np.uint8)
#       #     if pred_lung_loc_mask.sum() > 0:
#       #         cv2.imwrite(des_path_mask / f"{raw_id}_{i}.jpg", pred_lung_loc_mask * 255)

#       # for resume only save when mask is created
#       self.save_csv(bnid, res_slice)

#   # Cleaning
#   def clean(self, bnid: str, is_res: bool = False):
#       # csv remove
#       try:
#         os.remove(self.csv_dir / f"{bnid}.csv")
#       except FileNotFoundError:
#           pass
#       clean_paths = [
#           self.mask_dir,
#       ]
#       for clean_path in clean_paths:
#           if os.path.exists(clean_path / bnid):
#               shutil.rmtree(clean_path / bnid)

## Runtime functions

In [44]:
# Deployment functions
def unzip_study(study_zip_path: str, src_path: str):
    study_name = os.path.basename(study_zip_path).split(".zip")[0]
    study_path = os.path.join(src_path, study_name)
    if not os.path.exists(study_path):
        os.makedirs(study_path)

    with zipfile.ZipFile(study_zip_path, 'r') as zip_ref:
        zip_ref.extractall(study_path)

    return study_path

def get_dicoms(study_path: str):
    '''
    Get all dicoms path from study_path, return a full image paths list.
    '''

    dicom_paths = []
    for root, _, files in os.walk(study_path):
        for file in files:
            dicom_path = os.path.join(root, file)
            dicom_paths.append(dicom_path)

    return dicom_paths

def sorted_dicom(dicom_paths: List[str], is_filter: bool = True):
    paths = []
    for dicom_path in dicom_paths:
        raw_data = pydicom.dcmread(dicom_path, force=True)

        valid = True
        if is_filter:
            try:
                wc = min(raw_data.WindowCenter)
                ww = max(raw_data.WindowWidth)
                valid = -200 >= wc and wc >= -650 and 1000 <= ww and ww <= 1600
            except:
                valid = False

            if not valid:
                continue

        try:
            img_data = np.array(raw_data.pixel_array)
            if img_data.shape != (512, 512):
                valid = False

            series_number = raw_data.SeriesNumber
            instance_number = raw_data.InstanceNumber
        except:
            valid = False

        if not valid:
            continue

        paths.append(
            (dicom_path, f"{series_number:02d}_{instance_number:04d}"))

    if is_filter and len(paths) <= 100:
        return sorted_dicom(dicom_paths, is_filter=False)

    def key(element):
        return element[1]

    return sorted(paths, key=key)

def check_dicom(dicom_path):
    '''
    Check if dicom_path is ct image dicom without any wrong tag. Return
    '''
    try:
        dcm = pydicom.dcmread(dicom_path)
        dcm_tag = str(dcm.dir)

        contains = ''
        status = True

        # Check "Patient Protocol" tag
        if "Patient Protocol" in dcm_tag:
            status = False
            contains += "Patient Protocol"

        # Check "Dose Report" tag
        if "Dose Report" in dcm_tag:
            status = False
            if len(contains) != 0:
                contains += ", "
            contains += "Dose Report"

        # Check "pdf" tag
        if "pdf" in dcm_tag or "PDF" in dcm_tag:
            status = False
            if len(contains) != 0:
                contains += ", "
            contains += "PDF"

        if not status:
            return False, "Contain " + contains

        # Check "Pixel Array" length
        pixel_array = dcm.pixel_array
        if len(pixel_array) != 512:
            return False, "Length of pixel array is not 512"

        # Check "PixelSpacing"
        pixel_spacing = dcm.get("PixelSpacing", "-1")
        if pixel_spacing == "-1":
            return False, "Don't have pixel spacing"

        # Check "Modality" tag
        modality = dcm.Modality
        if modality != "CT":
            return False, "Is not CT Modality"

        return True, ""

    except Exception as e:
        return False, str(e)

def read_input(dicom_path: str):
    '''
    Read raw dicom data from dicom_path, return pixel_array of that dicom, an unique_id and pixel spacing tag.
    '''
    raw_data = pydicom.dcmread(dicom_path, force=True)

    # series_number = raw_data.SeriesNumber
    # instance_number = raw_data.InstanceNumber

    wc = raw_data.WindowCenter
    ww = raw_data.WindowWidth
    if isinstance(wc, pydicom.multival.MultiValue):
        wc = wc[0]
    if isinstance(ww, pydicom.multival.MultiValue):
        ww = ww[0]

    intercept = raw_data.RescaleIntercept
    slope = raw_data.RescaleSlope

    pixel_spacing = raw_data.get('PixelSpacing', np.array([1, 1]))
    raw_data = np.array(raw_data.pixel_array)

    return raw_data, pixel_spacing, intercept, slope, wc, ww

## Pipeline

In [23]:
def save_gray(filename, array):
    image = array.copy()
    image = (image - image.min()) / image.max()
    image = (image * 255).astype(np.uint8)
    return cv2.imwrite(filename, image[..., None])
    

In [49]:
# Pipeline running
from collections import defaultdict
from copy import deepcopy
def pipeline_infer(bnid, data_dir: Path, label_dir: Path, cache_dir: Path, force=True, pbar: tqdm =None):
    mask_dir = label_dir / 'mask' / bnid
    csv_dir = label_dir / 'csv'
    input_cache_dir = cache_dir / bnid
    if not os.path.isdir(input_cache_dir):
        input_cache_dir.mkdir(parents=True, exist_ok=True)
    dicom_paths = get_dicoms(data_dir / bnid)
    raw_df = pd.read_csv(csv_dir / f"{bnid}.csv")
    
    df = raw_df[raw_df['mask_id'].notna()]
    dicom_paths = sorted(dicom_paths)
    # print(len(dicom_paths))
    for dicom_path in dicom_paths:
        # is_ct, _ = check_dicom(dicom_path)
        # if not is_ct:
        #     continue
        dicom_file = dicom_path.split("/")[-1].split(".")[0] + ".jpg"
        cache_file = input_cache_dir / dicom_file

        slice_idx = int(dicom_path.split('.')[0].split('_')[-1])
        slice_node = raw_df[raw_df['slice_id'] == slice_idx]
        raw_id = slice_node['raw_id'].tolist()[0]
        node_df = df[df['slice_id'] == slice_idx].copy()
        bboxes = dict()
        if len(node_df) > 0:
            for index, node in node_df.iterrows(): 

                name = f"slice_{slice_idx:04d}_{node['mask_id']}.jpg"
                # print(name)
                label_mask = cv2.imread(mask_dir / name)[..., 0].astype(float) / 255
                # print(label_mask.shape, np.unique(label_mask))
                _, slice_bbox = find_bbox(label_mask, ratio=True, ssize=label_mask.shape[:2][::-1])
                # print
                bboxes[name[:-4]] = deepcopy(slice_bbox[0])
        # if pbar:
        #     pbar.set_description_str(f"{bnid}-{raw_id}".ljust(30), refresh=False)
        dcom = pydicom.dcmread(dicom_path, force=True)
        data = dcom.pixel_array
        # print(pixel_spacing, intercept, slope, wc, ww)
        pbar.set_description_str("Save to cache:" + str(save_gray(cache_file, data.copy())))
        # print(bboxes)
        # system.infer_2D(raw_data, bnid, slice_idx, raw_id, pixel_spacing, intercept, slope, wc, ww, 
        #                 label_bboxes=bboxes if (len(bboxes) > 0) and force else None, pbar=pbar)
    # cancer_pred = system.infer_3D(bnid)

# Demo

## Testing

Run all above cell, then run the next cell to initialize the inference model.

In [15]:
# torch.serialization.add_safe_globals({'src.models.model_2D.segcls.net.segcls.SegClsNet': SegClsNet})

In [16]:
# net = torch.load("checkpoints/segcls_struct.pth", weights_only=False)

In [17]:
# net.load_state_dict(torch.load("checkpoints/segcls.pth"))

In [18]:
# from datetime import datetime
# now = datetime.now()
# output_dir = Path(f"outputs/4.3_model_teacher/{now.year}_{now.month}_{now.day}-{now.hour}_{now.minute}_{now.second}")
# system = KCCTPipeline(net=net,
#                       device="cuda:0",
#                       output_dir=output_dir)

Run next cell to specify the {bnid} data folder and infer all dcm images inside.

In [50]:
# bnid = "CTB-4"
data_dir = Path(r"E:/AwesomeConformalPrediction/data/data_conformal/export/dicom")
label_dir = Path(r"E:/AwesomeConformalPrediction/data/data_conformal/export/ground_truth")
cache_dir = Path(r"E:/AwesomeConformalPrediction/data/data_conformal/export/cache")
# csv_dir = Path("")
# pipeline_infer(system, bnid, data_dir, label_dir, force=True)
with tqdm(os.listdir(data_dir), desc="Inference", bar_format="{l_bar} {bar}| {desc}") as pbar:
    for bnid in pbar:
#         # try:
        pipeline_infer(bnid, data_dir, label_dir, cache_dir, force=True, pbar=pbar)
#         # except Exception as e:
#         #     print(f"Error: {e}")
#         #     system.clean(bnid)
#         #     continue

Save to cache:False:   0%|           | Save to cache:False


AttributeError: Unable to decode the pixel data as the dataset's 'file_meta' has no (0002,0010) 'Transfer Syntax UID' element

Result save at "model_outputs/{bnid}/":
- "model_outputs/{bnid}/mask" -> 2D segmentation mask
- "model_outputs/{bnid}/csv" -> 2D classifications

In [20]:
# system.clean(bnid)

Run next cell to clean model outputs.